# Neural Net Architechture of Rotten Tomatoes

In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import one_hot

C:\Users\Aman Agarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Importing the dataset

In [2]:
df = pd.read_csv("train.tsv", delimiter = '\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df['Sentiment'].unique()

array([1, 2, 3, 4, 0], dtype=int64)

##### Getting all the phrases from the dataset into a single list

In [4]:
text = []
for t in df['Phrase']:
    text.append(t)

In [5]:
print(text[1])

A series of escapades demonstrating the adage that what is good for the goose


##### Converting the list that we made into number using one hot encoding so that we can feed our list to a neural network

In [6]:
enc = []
for index in text:
    enc.append(one_hot(index,100))

In [7]:
print(enc[1])
print(len(enc))

[24, 39, 51, 74, 31, 9, 16, 39, 11, 32, 69, 68, 9, 9]
156060


In [8]:
x = enc
y = df['Sentiment']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
print(len(y))

156060


#### Spliting our data into training and testing data

In [11]:
x_train = x[:100000]
x_test = x[100000:]
len(x_train)

100000

## Now Buliding the architecture of our network

#### Since it is a kind of classification problem wherein we have to assign sentiment rating on the basis of the review received therefore we will use Sequential kind of model

#### 

In [12]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense
from keras.optimizers import Adam

#### Now we will make seperate lists corrosponding to each sentiment..and then append all those lists to a single list to be able to feed into our neural network

In [13]:
l0 = []
l1 = []
l2 = []
l3 = []
l4 = []
count =0
for i in x:
    if(y[count]==1):
        for j in range(0,len(i)):
            l1.append(i[j])
            
    elif(y[count]==2):
        for j in range(0,len(i)):
            l2.append(i[j])
            
    elif(y[count]==3):
        for j in range(0,len(i)):
            l3.append(i[j])
            
    elif(y[count]==4):
        for j in range(0,len(i)):
            l4.append(i[j])
            
    elif(y[count]==0):
        for j in range(0,len(i)):
            l0.append(i[j])
    count+=1
print(len(l1))
print(len(l2))
print(len(l3))
print(len(l4))
print(len(l0))

y_test = [1,2,3,4,0]

238127
401206
264665
92187
80513


#### We converted our lists to numpy array so that we can easily reshape it according to the need of the model

In [14]:
f = []
f.append(l1)
f.append(l2)
f.append(l3)
f.append(l4)
f.append(l0)

arr = np.array(f)
arr.reshape(-1,1)
from keras.utils import to_categorical
y_binary = to_categorical(y_test)

In [15]:
print(y_binary)

[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]


#### In order to feed out data to neural network we require data of same length..To do this pad_sequences function from keras library is used which will add 0s in the begg. of every vector to make it to the same lenght..To add 0s to last we have to write pad_sequences(x, length,padding = 'post)'

In [18]:
from keras.preprocessing import sequence
arr = sequence.pad_sequences(arr,80000)
from keras.optimizers import RMSprop

### Now we will compare the performance of different model..with LSTM and without LSTM

#### What LSTM does is that it will help the model to understand which words are important for the model and which are not

In [21]:
model=Sequential()
model.add(Embedding(100, 32))
model.add(LSTM(32))
model.add(Dense(5, activation='sigmoid'))
opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss = 'categorical_crossentropy',optimizer = opt ,metrics = ['accuracy'] )

In [22]:
model.fit(arr, y_binary, batch_size=16, epochs=4)

Epoch 1/4
5/5 [==============================] - 39s 8s/step - loss: 1.6105 - acc: 0.0000e+00
Epoch 2/4
5/5 [==============================] - 39s 8s/step - loss: 1.6051 - acc: 0.2000
Epoch 3/4
5/5 [==============================] - 39s 8s/step - loss: 1.6010 - acc: 0.8000
Epoch 4/4
5/5 [==============================] - 43s 9s/step - loss: 1.5967 - acc: 0.8000


In [23]:
model1=Sequential()
model1.add(Embedding(1000, 32))
model1.add(LSTM(32))
model1.add(Dense(5, activation='softmax'))

model1.compile(loss = 'categorical_crossentropy',optimizer = 'adam' ,metrics = ['accuracy'] )

In [24]:
model1.fit(arr, y_binary, batch_size=64, epochs=3)

Epoch 1/3
5/5 [==============================] - 41s 8s/step - loss: 1.6059 - acc: 0.2000
Epoch 2/3
5/5 [==============================] - 43s 9s/step - loss: 1.6022 - acc: 0.4000
Epoch 3/3
5/5 [==============================] - 41s 8s/step - loss: 1.5985 - acc: 0.8000


In [25]:
model2=Sequential()
model2.add(Dense(5, activation='sigmoid'))

model2.compile(loss = 'categorical_crossentropy',optimizer = 'adam' ,metrics = ['accuracy'] )

In [26]:
model2.fit(arr, y_binary, batch_size=128, epochs=3)

Epoch 1/3
5/5 [==============================] - 1s 109ms/step - loss: 7.0253 - acc: 0.2000
Epoch 2/3
5/5 [==============================] - 0s 3ms/step - loss: 9.9481 - acc: 0.2000
Epoch 3/3
5/5 [==============================] - 0s 3ms/step - loss: 9.9481 - acc: 0.2000


### It can be clearly observed that LSTM model gave much higher accuary and low loss
### The reason being it chucks out embeddings which are useless for our model